In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_ini=pd.read_csv('titanic.csv')

In [3]:
for feature in df_ini.columns[:]:
    if df_ini[feature].dtype!='object':        
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].median(),df_ini[feature])
    else:
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].value_counts().index[0],df_ini[feature])
df_aim=df_ini

In [4]:
def reol(feature1):
    IQR=df_aim[feature1].quantile(0.75)-df_aim[feature1].quantile(0.25)
    lbound=df_aim[feature1].quantile(0.25)-(IQR*1.5)
    ubound=df_aim[feature1].quantile(0.75)+(IQR*1.5)
    df_aim.loc[df_aim[feature1]<=lbound,feature1]=lbound
    df_aim.loc[df_aim[feature1]>=ubound,feature1]=ubound
df_aol=df_aim

In [5]:
for feature in df_aim.columns[:]:
    if df_aim[feature].dtype!='object':
        reol(feature)

In [6]:
df_aol_cont1=df_aol.select_dtypes(exclude=[object])

In [7]:
df_aol_cont2=df_aol_cont1.drop(columns='Pclass',axis=1) #Dropping categorical ordinal features

In [8]:
df_aol_cont3=df_aol_cont2.drop('Survived',axis=1)#varience threshold can be applied for independent features only

In [9]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df_aol_cont3)

VarianceThreshold(threshold=0)

In [10]:
const_features=[column for column in df_aol_cont3.columns\
               if column not in df_aol_cont3.columns[var_thres.get_support()]]

In [11]:
df_aol=df_aol.drop(const_features,axis=1)#Dropping constant features

In [12]:
df_aol_cont3=df_aol_cont3.drop(columns=const_features,axis=1)

In [13]:
X=df_aol_cont3
y=df_aol_cont2['Survived']

In [14]:
def pcc(dataset,threshold):
    dropset=set()
    cor_mat=dataset.corr()
    for i in range(len(cor_mat.columns)):
        for j in range(i):
            if abs(cor_mat.iloc[i,j])>=threshold:
                cor_feat=cor_mat.columns[i]
                dropset.add(cor_feat)
    return dropset 
dropset1=pcc(X,0.85)
print("The Features to be Dropped are...")
dropset1

The Features to be Dropped are...


set()

In [15]:
df_aol=df_aol.drop(columns=dropset1,axis=1)

In [16]:
def enc(feature1):
    key_labels=df_aol[feature1].unique()
    dict={k:i for i,k in enumerate(key_labels,0)}
    df_aol[feature1]=df_aol[feature1].map(dict)

In [17]:
for feature in df_aol.columns[:]:
    if df_aol[feature].dtype=='object':
        enc(feature)
df_aec=df_aol

In [18]:
from sklearn.feature_selection import mutual_info_classif
X=df_aec.drop(columns='Survived',axis=1)
y=df_aec['Survived']
mut_inf_data=mutual_info_classif(X,y,random_state=0)
mut_inf_modif=pd.Series(mut_inf_data,index=X.columns)
mut_inf_sorted=mut_inf_modif.sort_values(ascending=False)
dict=mut_inf_sorted.to_dict()

In [19]:
droplist=[]
for key in dict:
    if dict[key]<0.001:
        droplist.append(key)

In [20]:
df_aec=df_aec.drop(columns=droplist,axis=1)

In [21]:
df_aim_cate1=df_aim.select_dtypes(include=[object])#For imbalanced data check,include all categorical features+Dep. feature

In [22]:
df_aim_cate2=df_aim[['Pclass','Survived']].copy()#include categorical ordinal feature and
df_aim_cate=pd.concat([df_aim_cate1,df_aim_cate2],axis=1)#dependent feature in the new Data frame.

In [23]:
imb_chk_list=list()

In [24]:
for feature in df_aim_cate.columns[:]:
    length=int(len(df_aim_cate[feature].unique()))      
    if length <= 5:
        imb_chk_list.append(feature)

In [25]:
drop_list=[]#Add the independent features with single category, imbalanced data in this list(Difference of values<0.05 and >0.95)

In [26]:
df_aec=df_aec.drop(drop_list,axis=1)

In [27]:
original_list=['Survived','Pclass','Sex','SibSp','Cabin','Embarked']
log_list=['Fare']
expo_list=[]
sqroot_list=['PassengerId','Name','Ticket']
boxcox_list=[]

In [28]:
for feature in log_list:
    df_aec[feature]=np.log1p(df_aec[feature])

In [29]:
for feature in sqroot_list:
    df_aec[feature]=df_aec[feature]**(1/2)

In [30]:
df_atr=df_aec

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled=scaler.fit_transform(df_atr)

In [32]:
df_ast=pd.DataFrame(scaled,columns=df_atr.columns)

In [33]:
df_fin=df_ast

In [34]:
X=df_fin.drop(columns='Survived',axis=1)
y=df_fin['Survived']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,random_state=0)

In [37]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[145  23]
 [ 31  69]]
Accuracy Score : 0.7985074626865671
              precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       168
         1.0       0.75      0.69      0.72       100

    accuracy                           0.80       268
   macro avg       0.79      0.78      0.78       268
weighted avg       0.80      0.80      0.80       268



In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [40]:
n_estimators=[200,2000,10]
max_depth=[10,100,10]
max_features=['auto','sqrt','log2']
param_grid={'n_estimators':n_estimators,'max_depth':max_depth,
            'max_features':max_features,'criterion':['entropy','gini']}

In [41]:
grid_search=GridSearchCV(estimator=classifier,param_grid=param_grid,n_jobs=-1,
                         cv=10,verbose=2)

In [42]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  7.2min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [10, 100, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 2000, 10]},
             verbose=2)

In [43]:
best_grid=grid_search.best_estimator_
best_grid

RandomForestClassifier(criterion='entropy', max_depth=10, max_features='log2',
                       n_estimators=200)

In [44]:
prediction=best_grid.predict(X_test)

In [45]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[150  18]
 [ 30  70]]
Accuracy Score : 0.8208955223880597
              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86       168
         1.0       0.80      0.70      0.74       100

    accuracy                           0.82       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.82      0.82      0.82       268



In [46]:
# pip install optuna
import optuna
from sklearn.model_selection import cross_val_score
def objective(trial):
    n_estimators=trial.suggest_int('n_estimators',200,2000,10)
    max_depth=int(trial.suggest_float('max_depth',10,100,log=True))
    classifier=RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=3).mean()
    return score

In [47]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2021-07-27 22:47:42,154] A new study created in memory with name: no-name-360e4c83-a260-4165-9592-cd4f366fede0
[I 2021-07-27 22:47:43,422] Trial 0 finished with value: 0.7640514678558157 and parameters: {'n_estimators': 380, 'max_depth': 28.135937477695823}. Best is trial 0 with value: 0.7640514678558157.
[I 2021-07-27 22:47:45,078] Trial 1 finished with value: 0.7656540319583797 and parameters: {'n_estimators': 500, 'max_depth': 68.02276251867954}. Best is trial 1 with value: 0.7656540319583797.
[I 2021-07-27 22:47:47,931] Trial 2 finished with value: 0.7688591601635079 and parameters: {'n_estimators': 870, 'max_depth': 11.600609459578603}. Best is trial 2 with value: 0.7688591601635079.
[I 2021-07-27 22:47:54,330] Trial 3 finished with value: 0.7656540319583797 and parameters: {'n_estimators': 1980, 'max_depth': 20.54878013357474}. Best is trial 2 with value: 0.7688591601635079.
[I 2021-07-27 22:47:59,121] Trial 4 finished with value: 0.7608463396506875 and parameters: {'n_estimat

[I 2021-07-27 22:49:37,584] Trial 39 finished with value: 0.7624334200421158 and parameters: {'n_estimators': 450, 'max_depth': 14.618154983928024}. Best is trial 15 with value: 0.7752849002849002.
[I 2021-07-27 22:49:40,639] Trial 40 finished with value: 0.76885141830794 and parameters: {'n_estimators': 940, 'max_depth': 11.093798409436657}. Best is trial 15 with value: 0.7752849002849002.
[I 2021-07-27 22:49:42,600] Trial 41 finished with value: 0.767241112349808 and parameters: {'n_estimators': 600, 'max_depth': 12.09467330761597}. Best is trial 15 with value: 0.7752849002849002.
[I 2021-07-27 22:49:44,436] Trial 42 finished with value: 0.7704617242660721 and parameters: {'n_estimators': 560, 'max_depth': 10.007147225772847}. Best is trial 15 with value: 0.7752849002849002.
[I 2021-07-27 22:49:45,957] Trial 43 finished with value: 0.7688591601635079 and parameters: {'n_estimators': 420, 'max_depth': 11.55535011863012}. Best is trial 15 with value: 0.7752849002849002.
[I 2021-07-27 2

[I 2021-07-27 22:51:51,998] Trial 78 finished with value: 0.7704617242660721 and parameters: {'n_estimators': 1640, 'max_depth': 10.782436101081212}. Best is trial 71 with value: 0.7784977703455964.
[I 2021-07-27 22:51:56,715] Trial 79 finished with value: 0.7672565960609439 and parameters: {'n_estimators': 1430, 'max_depth': 11.423993613722677}. Best is trial 71 with value: 0.7784977703455964.
[I 2021-07-27 22:52:00,752] Trial 80 finished with value: 0.7608308559395516 and parameters: {'n_estimators': 1230, 'max_depth': 12.103792749411605}. Best is trial 71 with value: 0.7784977703455964.
[I 2021-07-27 22:52:04,135] Trial 81 finished with value: 0.7656385482472438 and parameters: {'n_estimators': 1080, 'max_depth': 10.026259145819175}. Best is trial 71 with value: 0.7784977703455964.
[I 2021-07-27 22:52:07,127] Trial 82 finished with value: 0.772064288368636 and parameters: {'n_estimators': 960, 'max_depth': 10.100559279173787}. Best is trial 71 with value: 0.7784977703455964.
[I 2021

In [48]:
best=study.best_params

In [49]:
best

{'n_estimators': 1030, 'max_depth': 10.680355489838405}

In [50]:
classifier=RandomForestClassifier(n_estimators=best['n_estimators'],max_depth=best['max_depth'])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [51]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[148  20]
 [ 32  68]]
Accuracy Score : 0.8059701492537313
              precision    recall  f1-score   support

         0.0       0.82      0.88      0.85       168
         1.0       0.77      0.68      0.72       100

    accuracy                           0.81       268
   macro avg       0.80      0.78      0.79       268
weighted avg       0.80      0.81      0.80       268



In [52]:
# pip install hyperopt
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
space={'criterion':hp.choice('criterion',['entropy','gini']),
       'max_depth':hp.quniform('max_depth',10,2000,10),
       'max_features':hp.choice('max_features',['auto','sqrt','log2',None]),
       'min_samples_leaf':hp.uniform('min_samples_leaf',0,0.5),
       'min_samples_split':hp.uniform('min_samples_split',0,1),
       'n_estimators':hp.choice('n_estimators',[10,50,300,750,1200,1300,1500])}

In [53]:
def objective(space):
    classifier=RandomForestClassifier(n_estimators=space['n_estimators'],max_depth=space['max_depth'],
               max_features=space['max_features'],min_samples_leaf=space['min_samples_leaf'],
               min_samples_split=space['min_samples_split'])
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=5).mean()
    return{'loss':-score,'status':STATUS_OK}

In [54]:
trials=Trials()
best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=80,trials=trials)
best

100%|████████████████████████████████████████████████| 80/80 [04:49<00:00,  3.62s/trial, best loss: -0.792941935483871]


{'criterion': 1,
 'max_depth': 680.0,
 'max_features': 2,
 'min_samples_leaf': 0.017348698841213372,
 'min_samples_split': 0.0780624275926618,
 'n_estimators': 0}

In [55]:
crit={0:'entropy',1:'gini'}
feat={0:'auto',1:'sqrt',2:'log2',3:None}
est={0:10,1:50,2:300,3:750,4:1200,5:1300,6:1500}

In [56]:
classifier=RandomForestClassifier(criterion=crit[best['criterion']],max_depth=best['max_depth'],
            max_features=feat[best['max_features']],min_samples_leaf=best['min_samples_leaf'],
            min_samples_split=best['min_samples_split'],n_estimators=est[best['n_estimators']])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[148  20]
 [ 28  72]]
Accuracy Score : 0.8208955223880597
              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86       168
         1.0       0.78      0.72      0.75       100

    accuracy                           0.82       268
   macro avg       0.81      0.80      0.81       268
weighted avg       0.82      0.82      0.82       268

